<a href="https://colab.research.google.com/github/ankitsrivastava637/AWS-speech-to-text-AllenNLP-sentiment-detection/blob/main/AWS_transciption_%26_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install boto3
#!pip install allennlp_models

import pandas as pd
import time
import boto3
from urllib.request import urlopen
import datetime
import json
from allennlp_models.pretrained import load_predictor




def amazon_transcribe_multi(audio_file_name,s3_link, max_speakers = None):
 
    if max_speakers > 10:
        raise ValueError("Maximum detected speakers is 10.")
 
    job_uri = s3_link
    job_name = (audio_file_name.split('.')[0]).replace(" ", "").replace("(", "").replace(")", "") + str(time.time() * 1000) 
  
    # check if name is taken or not
    #job_name = check_job_name(job_name)
  
    if max_speakers != -1:
        transcribe.start_transcription_job(
          TranscriptionJobName=job_name,
          Media={'MediaFileUri': job_uri},
          MediaFormat=audio_file_name.split('.')[1],
          IdentifyLanguage = True,
          #LanguageOptions = ["en-IN", "hi-IN"],
          Settings = {'ShowSpeakerLabels': True,
                  'MaxSpeakerLabels': max_speakers
                  }
          )
    else: 
        transcribe.start_transcription_job(
           TranscriptionJobName=job_name,
           Media={'MediaFileUri': job_uri},
           MediaFormat=audio_file_name.split('.')[1],
           IdentifyLanguage = True,
           #LanguageOptions = ["en-IN", "hi-IN"],
           Settings = {'ShowSpeakerLabels': True
                  }
           )    
  
    while True:
        result = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        if result['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            #print('not yet ready')
            break
        time.sleep(15)

    if result['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
        #print('your transcript is ready') 
        #data = pd.read_json(result['TranscriptionJob']['Transcript']['TranscriptFileUri'])
        transcript_uri = urlopen(result["TranscriptionJob"]["Transcript"]["TranscriptFileUri"])
        transcript_json = json.dumps(json.load(transcript_uri))
        print("transcription done")
    return transcript_json






def read_output(filename, transcript_json):


  # Create an output txt file
    print(filename+'.txt')
    with open(filename+'.txt','w') as w:

        data=json.loads(transcript_json)
        labels = data['results']['speaker_labels']['segments']
        speaker_start_times={}
      
        for label in labels:
            for item in label['items']:
                speaker_start_times[item['start_time']] = item['speaker_label']

        items = data['results']['items']
        lines = []
        line = ''
        time = 0
        speaker = 'null'
        i = 0

        # loop through all elements
        for item in items:
            i = i+1
            content = item['alternatives'][0]['content']

        # if it's starting time
            if item.get('start_time'):
                current_speaker = speaker_start_times[item['start_time']]
        
           # in AWS output, there are types as punctuation
            elif item['type'] == 'punctuation':
                line = line + content

           # handle different speaker
            if current_speaker != speaker:
                if speaker:
                    lines.append({'speaker':speaker, 'line':line, 'time':time})
                line = content
                speaker = current_speaker
                time = item['start_time']
          
            elif item['type'] != 'punctuation':
                line = line + ' ' + content
        lines.append({'speaker': speaker, 'line': line,'time': time})
        #lines.append({'line': line,'time': time})


        # sort the results by the time
        sorted_lines = sorted(lines,key=lambda k: float(k['time']))

        uncleaned_line = ''

        #write the lines in file 
        for line_data in sorted_lines:
                line = '[' + str(datetime.timedelta(seconds=int(round(float(line_data['time']))))) + '] ' + line_data.get('speaker') + ': ' + line_data.get('line')
                #line = '[' + str(datetime.timedelta(seconds=int(round(float(line_data['time']))))) + '] ' + ': ' + line_data.get('line')
                uncleaned_line = uncleaned_line + line + '\n\n'
                w.write(line + '\n\n')
        
        read = pd.DataFrame()

        #read the lines in a dataframe format
        for line in uncleaned_line:
        # look at line in loop
          read = read.append({'all_data' : line}, 
                ignore_index = True)

        
        
        #This section will give all the text conversations between all speakers at a single place in the variable 'conversation' 
        conversation = ''
        for i in range(len(read)):
          spk_1_str = str(read.values[i])

          #for speaker 0 
          try:
            spk1_str_f = spk_1_str.partition("spk_0")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass
          
          #for speaker 1 
          try:
            spk1_str_f = spk_1_str.partition("spk_1")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass
          
          #for speaker 2 : if he/she exists
          try:
            spk1_str_f = spk_1_str.partition("spk_2")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass

          #for speaker 3 : if he/she exists
          try:
            spk1_str_f = spk_1_str.partition("spk_3")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass  

          #for speaker 4 : if he/she exists
          try:
            spk1_str_f = spk_1_str.partition("spk_4")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass  

          #for speaker 5 : if he/she exists
          try:
            spk1_str_f = spk_1_str.partition("spk_5")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass   

          #for speaker 6 : if he/she exists
          try:
            spk1_str_f = spk_1_str.partition("spk_6")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass 

          #for speaker 7 : if he/she exists
          try:
            spk1_str_f = spk_1_str.partition("spk_7")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass

          #for speaker 8 : if he/she exists
          try:
            spk1_str_f = spk_1_str.partition("spk_8")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass

          #for speaker 9 : if he/she exists
          try:
            spk1_str_f = spk_1_str.partition("spk_9")[2].replace(":", "").replace("]", "").replace('\\n', "")
            spk1_str_f = spk1_str_f.rstrip(spk1_str_f[-1])
            conversation = conversation + spk1_str_f
          except:
            pass 

        #sentiment analysis with AllenNLP's library for english copnversations
        try:
          #sentiment analysis with AllenNLP's library for english conversations 
          predictor = load_predictor("roberta-sst")
          preds = predictor.predict(conversation)
        
          #overall positivity score 
          positivity = "Positivity score of the conversation : " + str(preds['probs'][0] * 100) + " %"
          #overall negativity score
          negativity = "Negativity score of the conversation : " + str(preds['probs'][1] * 100) + " %"

          #write the positivity score in the file
          w.write("\n" + "\n" + "\n" + "\n" + "\n" + positivity + "\n" + "\n")
          #write the negativity score in the file
          w.write(negativity)
        except:
          pass




if __name__ is "__main__":
    
    transcribe = boto3.client('transcribe',
                          aws_access_key_id = "#Your Access key",
                          aws_secret_access_key = "#Your secret access key",
                          region_name = "ap-south-1")
    audio_file_name = 'aws-hinglish-test.mp3'
    s3_link = "s3://hinglish-test/aws-hinglish-test.mp3"
    max_speakers = 10


    transcript_json = amazon_transcribe_multi(audio_file_name,s3_link, max_speakers)

    read_output(audio_file_name.split('.')[0].replace(" ", ""), transcript_json)    

transcription done
aws-hinglish-test.txt


roberta-rte is not a registered model.
lerc is not a registered model.
Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'


In [ ]:
predictor = load_predictor("roberta-sst")
preds = predictor.predict("প্রবল কম্পন কেঁপে উঠল অসম , উত্তর৷")
        
#overall positivity score 
positivity = "Positivity score of the conversation : " + str(preds['probs'][0] * 100) + " %"
#overall negativity score
negativity = "Negativity score of the conversation : " + str(preds['probs'][1] * 100) + " %"

print(positivity)


roberta-rte is not a registered model.
lerc is not a registered model.
Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'


Positivity score of the conversation : 60.99346876144409 %
